In [1]:
import pandas as pd
import numpy as np
from pydataset import data

In [2]:
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

1. Load the `mpg` dataset. Read the documentation for it, and use the data to answer these questions:

In [3]:
mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


    - On average, which manufacturer has the best miles per gallon?

In [4]:
#create a variable avg_mpg to get average of hwy and cty
avg_mpg = mpg[['hwy', 'cty']].agg('mean', axis = 1)
#add a column to represent the average mileage for all vehicles
mpg['average_mileage'] = avg_mpg
#set a variable to hold a series grouped by manufacturer showing each average mileage from highest to lowest
mpg_manufacturer = mpg.groupby('manufacturer').average_mileage.max().sort_values(ascending=False)
#return the manufacturer with best mpg
mpg_manufacturer.nlargest().head(1)

manufacturer
volkswagen    39.5
Name: average_mileage, dtype: float64

    - How many different manufacturers are there?

In [5]:
#alternate solution
#len(mpg_manufacturer)
len(mpg['manufacturer'].value_counts())

15

    - How many different models are there?

In [6]:
#alternate solution
#len(mpg['model'].value_counts())
len(mpg.groupby('model'))

38

    - Do automatic or manual cars have better miles per gallon?

In [7]:
trans_avg_mpg = mpg.groupby(['trans']).average_mileage.agg(['mean'])
auto = trans_avg_mpg.iloc[:8].mean()
manual = trans_avg_mpg.iloc[8:].mean()
print(f'{manual}: manual')
print(f'{auto}: auto')

mean    21.664247
dtype: float64: manual
mean    20.773042
dtype: float64: auto


2. Joining and Merging

    Copy the `users` and `roles` dataframes from the examples above. What do you think a `right` join would look like? An `outer` join? What happens if you drop the foreign keys from the dataframes and try to merge them?

In [8]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [9]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [10]:
# a right join would exclude the values from users that dont have users.role_id = roles.id
# the commenter would not have values for users.id and users.name
pd.merge(users,roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [11]:
# an outer join will include all values from each including nulls/NaNs
pd.merge(users,roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [12]:
# if you dropped the foreign keys from the dataframes they wouldn't be able to merge
# because they have no place to merge from

3. Getting data from SQL databases

        a. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

In [13]:
from env import host, user, password

get_db_url = f'mysql+pymysql://{user}:{password}@{host}/employees'

        b. Use your function to obtain a connection to the employees database.

In [14]:
get_db_url = f'mysql+pymysql://{user}:{password}@{host}/employees'

        c. Once you have successfully run a query:
            - Intentionally make a typo in the database url. What kind of error message do you see?

In [15]:
#OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'darden_1039'@'%' to database 'employes'")
#(Background on this error at: http://sqlalche.me/e/e3q8)

            - Intentionally make an error in your SQL query. What does the error message look like?

In [16]:
#ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FRO employees LIMIT 5 OFFSET 50' at line 1")
#[SQL: SELECT * FRO employees LIMIT 5 OFFSET 50]
#(Background on this error at: http://sqlalche.me/e/f405)

        d. Read the employees and titles tables into two separate dataframes

In [17]:
employees = pd.read_sql('SELECT * FROM employees', get_db_url)

In [18]:
titles = pd.read_sql('SELECT * FROM titles', get_db_url)

        e. Visualize the number of employees with each title.

In [21]:
number_of_emp_title = titles.groupby(['title']).emp_no.agg(['count'])
number_of_emp_title

,count
title,
Assistant Engineer,15128
Engineer,115003
Manager,24
Senior Engineer,97750
Senior Staff,92853
Staff,107391
Technique Leader,15159


        f. Join the employees and titles dataframes together.

In [22]:
employees_titles = pd.merge(employees,titles, left_on='emp_no', right_on='emp_no', how='inner')
employees_titles

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


        g. Visualize how frequently employees change titles.

In [24]:
title_change = titles.emp_no.value_counts()

title_change

103876    3
467421    3
204120    3
105515    3
262553    3
         ..
201714    1
254956    1
250858    1
248809    1
10245     1
Name: emp_no, Length: 300024, dtype: int64

        h. For each title, find the hire date of the employee that was hired most recently with that title.

In [25]:
employees_titles.groupby(['title']).hire_date.agg(['max'])

,max
title,
Assistant Engineer,1999-12-24
Engineer,2000-01-28
Manager,1992-02-05
Senior Engineer,2000-01-01
Senior Staff,2000-01-13
Staff,2000-01-12
Technique Leader,1999-12-31


        i. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL and python/pandas code)

In [31]:
departments = pd.read_sql('SELECT * FROM departments', get_db_url)
dept_emp = pd.read_sql('SELECT * FROM dept_emp', get_db_url)

In [34]:
titles_dept_emp = pd.merge(\
titles, dept_emp,\
left_on='emp_no', right_on='emp_no', how='inner')

In [45]:
titles_dept_emp_dept = pd.merge(\
titles_dept_emp, departments,\
left_on='dept_no', right_on='dept_no', how='inner')

In [44]:
pd.crosstab(titles_dept_emp_dept.title, titles_dept_emp_dept.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,298,7769,0,0,0,6445,1831,378,0
Engineer,2362,58135,0,0,0,49649,13852,2986,0
Manager,4,2,2,2,2,4,4,2,2
Senior Engineer,2027,49326,0,0,0,42205,11864,2570,0
Senior Staff,13925,1247,12139,12274,13940,1270,0,11637,36191
Staff,16150,1424,13929,14342,16196,1478,0,13495,41808
Technique Leader,309,7683,0,0,0,6557,1795,393,0


4. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [46]:
get_db_url = f'mysql+pymysql://{user}:{password}@{host}/chipotle'

    - What is the total price for each order?

In [47]:
#create the orders dataframe from the orders table from the chipotle db
orders = pd.read_sql('SELECT * FROM orders', get_db_url)
# replaced the $ in item_price and changed the datatype to float
orders['item_price'] = (orders['item_price'].str.replace('$','')).astype('float')
#create a total_price column that takes the item_price * quantity
orders['total_price'] = orders.quantity * orders.item_price
orders[['total_price']]

,total_price
0,2.39
1,3.39
2,3.39
3,2.39
4,33.96
...,...
4617,11.75
4618,11.75
4619,11.25
4620,8.75


    - What are the most popular 3 items?

In [48]:
orders.item_name.value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

    - Which item has produced the most revenue?

In [49]:
orders.groupby('item_name').total_price.agg('sum').nlargest(1)

item_name
Chicken Bowl    8044.63
Name: total_price, dtype: float64